In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np

cuda:0


In [2]:
import torch
import torch.nn as nn

# 定义一个简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu4 = nn.ReLU()

        # 使用Xavier初始化来初始化权重
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)

        # 初始化偏置为零
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.bias, 0)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        return out
        
# 定义输入、隐藏层和输出的维度
input_size = 10
hidden_size = 20
output_size = 10

# 创建一个SimpleNN模型



In [3]:
class customloss(nn.Module):
    def __init__(self, theta):
        super(customloss, self).__init__()
        self.theta = theta
    def forward(self, output_list, y_list):
        theta = torch.tensor(self.theta, dtype= torch.float32).to(device) 
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)))  
            loss += torch.sum(-y * torch.log(prob) - (1-y) * torch.log(1-prob)) 
            index += 1  
        return loss / (len(y_list)/100)

In [4]:
CONTEXT_ARRAY = np.load('features1.npy')
theta = np.load('theta.npy')
PURCHASE_LIST = np.load('purchase.npy')

In [5]:
feature_list = []
purchase_list = []

model = SimpleNN(input_size, hidden_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)
for t in range(0,10000):
    context = CONTEXT_ARRAY[t]

    purchase_list.append(PURCHASE_LIST[t])

    feature_list.append(np.array(context))

    if t%100 == 99:
        loss_function = customloss(theta)
        epochs = 1
        for epoch in range(epochs):
            
            output_list = [model(torch.tensor(a,dtype = torch.float32).to(device)) for a in feature_list]
            loss = loss_function(output_list, purchase_list)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 2411.39892578125
epoch 0, loss 2411.3798828125
epoch 0, loss 2411.36767578125
epoch 0, loss 2411.357421875
epoch 0, loss 2411.349853515625
epoch 0, loss 2411.34375
epoch 0, loss 2411.34033203125
epoch 0, loss 2411.337158203125
epoch 0, loss 2411.334716796875
epoch 0, loss 2411.333251953125
epoch 0, loss 2411.332275390625
epoch 0, loss 2411.33154296875
epoch 0, loss 2411.330810546875
epoch 0, loss 2411.33056640625
epoch 0, loss 2411.330078125
epoch 0, loss 2411.329833984375
epoch 0, loss 2411.32958984375
epoch 0, loss 2411.329345703125
epoch 0, loss 2411.329345703125
epoch 0, loss 2411.3291015625
epoch 0, loss 2411.3291015625
epoch 0, loss 2411.3291015625
epoch 0, loss 2411.3291015625
epoch 0, loss 2411.328857421875
epoch 0, loss 2411.32861328125
epoch 0, loss 2411.328857421875
epoch 0, loss 2411.32861328125
epoch 0, loss 2411.34033203125
epoch 0, loss 2411.353271484375
epoch 0, loss 2411.365478515625
epoch 0, loss 2411.376708984375
epoch 0, loss 2411.387451171875
epoch 0,

In [6]:
problist  = np.load('prob_list.npy')
def hhh(i):
    ff = model(torch.tensor(CONTEXT_ARRAY[i],dtype = torch.float32).to(device))
    ff = ff.cpu().detach().numpy()
    u = (ff @ theta.T ).reshape(1,-1)
    prob = np.exp(u) / (np.sum(np.exp(u)))
    
    print(problist[i])
    print(prob)

hhh(100)   

[0.07958204 0.22451541 0.24934586 0.08885355 0.05108825 0.08780105
 0.09688146 0.12193237]
[[0.12506875 0.12487867 0.12472919 0.12538411 0.12547892 0.1245039
  0.12506193 0.12489452]]


In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, output_list, y_list, theta):
        theta = torch.tensor(theta, dtype=torch.float32).to(output_list[0].device)
        loss = 0
        for output, y in zip(output_list, y_list):
            v = torch.matmul(output, theta)
            prob = torch.exp(v) / torch.sum(torch.exp(v))
            loss += torch.sum(-y * torch.log(prob) - (1-y) * torch.log(1-prob))
        return loss / len(y_list)